<a href="https://colab.research.google.com/github/garridoeduardo1000-afk/Simulacion-2-Eduardo-Gonzalez-/blob/main/lineadosservidores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np
from random import expovariate
import time

def simular_empresa_servicios_corregido(lambda_llegadas, mu1, mu2, num_clientes=100000):

    start_time = time.time()

    # Inicialización de variables
    t = 0.0
    n1 = 0  # clientes en estación 1 (incluyendo el en servicio)
    n2 = 0  # clientes en estación 2 (incluyendo el en servicio)
    N_S = 0 # clientes que han salido del sistema

    # Tiempos de próximos eventos
    t_LL = expovariate(lambda_llegadas)
    t1 = float('inf')
    t2 = float('inf')


    cola_estacion1 = []  # Clientes esperando o en servicio en estación 1
    cola_estacion2 = []  # Clientes esperando o en servicio en estación 2


    tiempos_sistema = []
    tiempos_fila1 = []   # Tiempo esperando antes de estación 1
    tiempos_fila2 = []   # Tiempo esperando antes de estación 2
    tiempos_servicio1 = [] # Tiempo en servicio estación 1
    tiempos_servicio2 = [] # Tiempo en servicio estación 2


    ultimo_cambio = 0.0
    area_fila1 = 0.0
    area_fila2 = 0.0
    n1_fila_prev = 0
    n2_fila_prev = 0

    print(f"===  ===")
    print(f"λ = {lambda_llegadas} clientes/hora")
    print(f"μ₁ = {mu1} clientes/hora")
    print(f"μ₂ = {mu2} clientes/hora")
    print("=" * 40)

    cliente_id = 0

    while N_S < num_clientes:
        # CASE 1: Llegada
        if t_LL <= t1 and t_LL <= t2:
            dt = t_LL - ultimo_cambio
            area_fila1 += dt * n1_fila_prev
            area_fila2 += dt * n2_fila_prev
            ultimo_cambio = t_LL

            t = t_LL
            n1 += 1

            # Cliente llega al sistema
            cliente = {
                'id': cliente_id,
                'llegada_sistema': t,
                'inicio_servicio1': None,
                'fin_servicio1': None,
                'inicio_servicio2': None,
                'fin_servicio2': None
            }
            cola_estacion1.append(cliente)
            cliente_id += 1

            # Actualizar número en fila (clientes esperando)
            n1_fila_prev = max(0, n1 - 1)  # Excluye el que está en servicio

            # Si es el primer cliente, iniciar servicio
            if n1 == 1:
                # El cliente que acaba de llegar empieza servicio inmediatamente
                cliente_servicio = cola_estacion1[0]
                cliente_servicio['inicio_servicio1'] = t
                t1 = t + expovariate(mu1)

            t_LL = t + expovariate(lambda_llegadas)

        # CASE 2: Servicio estación 1
        elif t1 <= t_LL and t1 <= t2:
            dt = t1 - ultimo_cambio
            area_fila1 += dt * n1_fila_prev
            area_fila2 += dt * n2_fila_prev
            ultimo_cambio = t1

            t = t1
            n1 -= 1

            if cola_estacion1:
                # El cliente que estaba en servicio termina
                cliente_terminado = cola_estacion1.pop(0)
                cliente_terminado['fin_servicio1'] = t

                # Calcular tiempo en fila 1 (espera antes de servicio 1)
                if cliente_terminado['inicio_servicio1'] is not None:
                    tiempo_espera1 = cliente_terminado['inicio_servicio1'] - cliente_terminado['llegada_sistema']
                    tiempos_fila1.append(tiempo_espera1)

                # Calcular tiempo de servicio 1
                if cliente_terminado['inicio_servicio1'] is not None and cliente_terminado['fin_servicio1'] is not None:
                    tiempo_serv1 = cliente_terminado['fin_servicio1'] - cliente_terminado['inicio_servicio1']
                    tiempos_servicio1.append(tiempo_serv1)

                # Cliente pasa a estación 2
                n2 += 1
                cliente_terminado['llegada_fila2'] = t  # Llega a la fila de estación 2
                cola_estacion2.append(cliente_terminado)

                # Actualizar fila 2
                n2_fila_prev = max(0, n2 - 1)  # Excluye el que está en servicio

                # Si es el primer cliente en estación 2, iniciar servicio
                if n2 == 1:
                    cliente_servicio2 = cola_estacion2[0]
                    cliente_servicio2['inicio_servicio2'] = t  # Inicia servicio inmediatamente
                    t2 = t + expovariate(mu2)

            # Programar próximo servicio estación 1
            if n1 > 0 and cola_estacion1:
                siguiente_cliente = cola_estacion1[0]
                siguiente_cliente['inicio_servicio1'] = t
                t1 = t + expovariate(mu1)
                n1_fila_prev = max(0, n1 - 1)
            else:
                t1 = float('inf')
                n1_fila_prev = 0

        # CASE 3: Servicio estación 2
        else:
            dt = t2 - ultimo_cambio
            area_fila1 += dt * n1_fila_prev
            area_fila2 += dt * n2_fila_prev
            ultimo_cambio = t2

            t = t2
            n2 -= 1
            N_S += 1

            if cola_estacion2:
                # El cliente que estaba en servicio termina
                cliente_terminado = cola_estacion2.pop(0)
                cliente_terminado['fin_servicio2'] = t

                # Calcular tiempo en fila 2 (espera antes de servicio 2)
                if cliente_terminado['inicio_servicio2'] is not None and cliente_terminado['llegada_fila2'] is not None:
                    tiempo_espera2 = cliente_terminado['inicio_servicio2'] - cliente_terminado['llegada_fila2']
                    tiempos_fila2.append(tiempo_espera2)

                # Calcular tiempo de servicio 2
                if cliente_terminado['inicio_servicio2'] is not None and cliente_terminado['fin_servicio2'] is not None:
                    tiempo_serv2 = cliente_terminado['fin_servicio2'] - cliente_terminado['inicio_servicio2']
                    tiempos_servicio2.append(tiempo_serv2)

                # Calcular tiempo total en sistema
                tiempo_total = t - cliente_terminado['llegada_sistema']
                tiempos_sistema.append(tiempo_total)

            # Actualizar fila 2
            n2_fila_prev = max(0, n2 - 1)

            # Programar próximo servicio estación 2
            if n2 > 0 and cola_estacion2:
                siguiente_cliente = cola_estacion2[0]
                siguiente_cliente['inicio_servicio2'] = t
                t2 = t + expovariate(mu2)
            else:
                t2 = float('inf')

    # Cálculos finales
    tiempo_total_simulacion = t

    W = np.mean(tiempos_sistema) if tiempos_sistema else 0
    Wq1 = np.mean(tiempos_fila1) if tiempos_fila1 else 0
    Wq2 = np.mean(tiempos_fila2) if tiempos_fila2 else 0
    S1 = np.mean(tiempos_servicio1) if tiempos_servicio1 else 0
    S2 = np.mean(tiempos_servicio2) if tiempos_servicio2 else 0

    Lq1 = area_fila1 / tiempo_total_simulacion
    Lq2 = area_fila2 / tiempo_total_simulacion

    end_time = time.time()



    print(f"\n--- TIEMPOS PROMEDIO (minutos) ---")
    print(f"Tiempo en sistema: {W*60:.2f} min")
    print(f"Tiempo en fila 1: {Wq1*60:.2f} min")
    print(f"Tiempo en fila 2: {Wq2*60:.2f} min")


    return W, Wq1, Wq2, Lq1, Lq2

# Ejecutar
if __name__ == "__main__":
    lambda_llegadas = 10.0
    mu1 = 12.0
    mu2 = 15.0

    print("Ejecutando versión corregida...")
    W, Wq1, Wq2, Lq1, Lq2 = simular_empresa_servicios_corregido(
        lambda_llegadas=lambda_llegadas,
        mu1=mu1,
        mu2=mu2,
        num_clientes=50000  # Reducido para prueba más rápida
    )

Ejecutando versión corregida...
=== SIMULACIÓN CORREGIDA ===
λ = 10.0 clientes/hora
μ₁ = 12.0 clientes/hora
μ₂ = 15.0 clientes/hora

--- TIEMPOS PROMEDIO (minutos) ---
Tiempo en sistema: 41.50 min
Tiempo en fila 1: 24.99 min
Tiempo en fila 2: 7.51 min


comparando con un ejemplo analitico

Estación 1:$$\rho_1 = \frac{\lambda_1}{\mu_1} = \frac{10}{12} = \frac{5}{6} \approx 0.8333$$
Estación 2: La tasa de llegada a la Estación 2 ($\lambda_2$) es igual a la tasa de llegada al sistema ($\lambda_1 = 10$).$$\rho_2 = \frac{\lambda_2}{\mu_2} = \frac{10}{15} = \frac{2}{3} \approx 0.6667$$
Tiempo Promedio de Espera en la Fila ($W_q$) para cada EstaciónPara un sistema $M/M/1$, el tiempo promedio de espera en la fila es:$$W_q = \frac{\lambda}{\mu(\mu - \lambda)}$$
Estación 1 ($W_{q1}$):$$W_{q1} = \frac{10}{12(12 - 10)} = \frac{10}{12(2)} = \frac{10}{24} = \frac{5}{12} \text{ horas}$$En minutos: $(\frac{5}{12} \text{ horas}) \times (60 \text{ min/hora}) = 25 \text{ minutos}$.Estación 2 ($W_{q2}$):$$W_{q2} = \frac{10}{15(15 - 10)} = \frac{10}{15(5)} = \frac{10}{75} = \frac{2}{15} \text{ horas}$$En minutos: $(\frac{2}{15} \text{ horas}) \times (60 \text{ min/hora}) = 8 \text{ minutos}$.
Tiempo Promedio en Servicio ($W_{s}$) para cada EstaciónPara un sistema $M/M/1$, el tiempo promedio en el sistema (espera en fila + servicio) es:$$W_s = W_q + \frac{1}{\mu} \quad \text{o} \quad W_s = \frac{1}{\mu - \lambda}$$Estación 1 ($W_{s1}$):$$W_{s1} = \frac{1}{12 - 10} = \frac{1}{2} \text{ horas}$$
En minutos: $(\frac{1}{2} \text{ horas}) \times (60 \text{ min/hora}) = 30 \text{ minutos}$.Estación 2 ($W_{s2}$):$$W_{s2} = \frac{1}{15 - 10} = \frac{1}{5} \text{ horas}$$En minutos: $(\frac{1}{5} \text{ horas}) \times (60 \text{ min/hora}) = 12 \text{ minutos}$.
Para un sistema en serie, los tiempos totales son simplemente la suma de los tiempos en cada estación.A. Tiempo Promedio Total que un cliente pasa formado en las filas ($\boldsymbol{W_q}$)$$W_q \text{ Total} = W_{q1} + W_{q2}$$$$W_q \text{ Total} = \frac{5}{12} \text{ horas} + \frac{2}{15} \text{ horas}$$Para sumar, usamos el mínimo común múltiplo (MCM) de 12 y 15, que es 60:$$W_q \text{ Total} = \frac{5 \times 5}{60} + \frac{2 \times 4}{60} = \frac{25 + 8}{60} = \frac{33}{60} \text{ horas}$$Simplificando la fracción:$$W_q \text{ Total} = \frac{11}{20} \text{ horas}$$Respuesta en minutos: $(\frac{11}{20} \text{ horas}) \times (60 \text{ min/hora}) = 11 \times 3 = \mathbf{33 \text{ minutos}}$.B. Tiempo Promedio Total que un cliente pasa en el sistema ($\boldsymbol{W_s}$)$$W_s \text{ Total} = W_{s1} + W_{s2}$$$$W_s \text{ Total} = \frac{1}{2} \text{ horas} + \frac{1}{5} \text{ horas}$$Para sumar, usamos el MCM de 2 y 5, que es 10:$$W_s \text{ Total} = \frac{5}{10} + \frac{2}{10} = \frac{7}{10} \text{ horas}$$Respuesta en minutos: $(\frac{7}{10} \text{ horas}) \times (60 \text{ min/hora}) = 7 \times 6 = \mathbf{42 \text{ minutos}}$.
El tiempo promedio que un cliente pasa formado en las filas (esperando servicio en la Estación 1 y luego en la Estación 2) es de 32 minutos.

El tiempo promedio que un cliente pasa en el sistema (espera en filas + tiempo de servicio en ambas estaciones) es de 41 minutos.
